# Volume Profile



In [ ]:
ticker = "SPY"
interval = "5m"
previous_days = 5
resolution_usd = 0.1



## Package management



In [ ]:
import datetime
import functools
from typing import List, Tuple, Union



In [ ]:
Version = Union[List[int], Tuple]



In [ ]:
def check_version(req:Version, version_str:str):
  v0, v1, v2 = version_str.split('.')

  result = False

  if int(v0) > req[0]:
    result = True
  elif int(v1) > req[1]:
    result = True
  elif int(v2) >= req[2]:
    result = True

  return result



Test function `check_version()` before using



In [ ]:
assert check_version([0, 9, 9], "0.10.0")



In [ ]:
assert check_version([0, 10, 0], "0.10.0")



How do I unload (reload) a Python module? https://stackoverflow.com/questions/437589/how-do-i-unload-reload-a-python-module



In [ ]:
import pandas as pd
if not check_version([1, 3, 5], pd.__version__):
  !pip install --upgrade pandas
  import importlib
  importlib.reload(pd)
  assert check_version([1, 3, 5], pd.__version__)



Unable to get stock data from yahoo with pandas_datareader

https://stackoverflow.com/questions/68327968/unable-to-get-stock-data-from-yahoo-with-pandas-datareader/68435757#68435757

Download Financial Dataset Using Yahoo Finance in Python | A Complete Guide https://www.analyticsvidhya.com/blog/2021/06/download-financial-dataset-using-yahoo-finance-in-python-a-complete-guide/



yfinance Library – A Complete Guide https://algotrading101.com/learn/yfinance-guide/


In [ ]:
import pandas as pd



In [ ]:
try:
  import yfinance as yf
except ModuleNotFoundError:
  !pip install yfinance
  !pip install yahoofinancials
  import yfinance as yf



In [ ]:
import numpy as np



Charting Candlestick_OHLC one minute bars with Pandas and Matplotlib https://stackoverflow.com/questions/41821916/charting-candlestick-ohlc-one-minute-bars-with-pandas-and-matplotlib

Candlestick Chart in Python (mplfinance, plotly, bokeh, bqplot and cufflinks) https://coderzcolumn.com/tutorials/data-science/candlestick-chart-in-python-mplfinance-plotly-bokeh#1

In [ ]:
try:
  import mplfinance as fplt
except ModuleNotFoundError:
  !pip install mplfinance
  import mplfinance as fplt



## Data period

In [ ]:
@functools.lru_cache()
def tz() -> str:
  return "America/New_York"



Most recent previous business day in Python https://stackoverflow.com/questions/2224742/most-recent-previous-business-day-in-python, https://stackoverflow.com/a/51528258



In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
@functools.lru_cache()
def USBday():
  return pd.tseries.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())



pandas.Series.dt.tz_convert https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.tz_convert.html

In [ ]:
end_date = pd.Timestamp.today(tz=tz())
start_date = end_date + USBday() * (-previous_days)
start_date



Python for Finance, Part I: Yahoo & Google Finance API, pandas, and matplotlib
https://www.learndatasci.com/tutorials/python-finance-part-yahoo-finance-api-pandas-matplotlib/

An Even Easier Guide to Getting Stock Data With Python
https://towardsdatascience.com/an-even-easier-guide-to-getting-stock-data-with-python-1a109df6b593



Google Finance API and Its Alternatives (Updated for 2021) https://rapidapi.com/blog/google-finance-api-alternatives/

## Getting the data



In [ ]:
spy = yf.Ticker(ticker)



In [ ]:
spy_historical = spy.history(start=start_date, end=end_date, interval=interval)



In [ ]:
spy_historical



## Plotting the chart



In [ ]:
import matplotlib.pyplot as plt



How can I customize mplfinance.plot? https://stackoverflow.com/questions/60599812/how-can-i-customize-mplfinance-plot

In [ ]:
def plot_candlestick(df:pd.DataFrame, ax=None):
  if ax is None:
    fig, ax = plt.subplots(figsize=(16, 9))

  fplt.plot(
      df,
      type='candle',
      style='charles',
      ylabel='Price ($)',
      ax=ax,
  );

  ax.grid(True)

  return ax



In [ ]:
ax = plot_candlestick(spy_historical)
ax.set_title(f'{ticker} from {start_date.date()} to {end_date.date()}');



## Volume Profile : daily

### `start_date`

Data of `start_date`



In [ ]:
def df_one_day(df:pd.DataFrame, date:pd.Timestamp) -> pd.DataFrame:
  return df.loc[(df.index >= date) & (df.index < date + USBday())]



In [ ]:
spy_historical_start_date = df_one_day(spy_historical, start_date)



In [ ]:
assert spy_historical_start_date.index.max().date() == start_date.date()



Histogram values of a Pandas Series https://stackoverflow.com/questions/13129618/histogram-values-of-a-pandas-series



Feature Request: Volume Profile on a chart with alpha argument #162 https://github.com/matplotlib/mplfinance/issues/162



In [ ]:
def get_bins(ser:pd.Series, resolution_usd:float):
  return np.arange(ser.min(), ser.max() + 1e-4, resolution_usd)



In [ ]:
bins_historical = get_bins(spy_historical["Close"], resolution_usd)



In [ ]:
def calc_histogram_close(df:pd.DataFrame, bins:np.ndarray) -> Tuple[np.ndarray]:
  return np.histogram(df["Close"], bins=bins, weights=df["Volume"])



In [ ]:
def plot_candlestick_vprofile(df_one_day:pd.DataFrame, ax=None, bins=None):
  ax = plot_candlestick(df_one_day, ax=ax)

  ax.set_title(f'{ticker} {df_one_day.index.min().date()}')

  width, y = calc_histogram_close(df_one_day, bins=bins)

  ax2 = ax.twiny()
  ax2.set_xlim((0, 1.2*width.max()))
  ax2.barh(y=y[:-1], width=width, height=resolution_usd, alpha=0.5);
  return ax, ax2



In [ ]:
plot_candlestick_vprofile(df_one_day(spy_historical, start_date), bins=bins_historical);



### all days



In [ ]:
def get_dates_tuple(df:pd.DataFrame) -> Tuple[pd.Timestamp]:
  return tuple(
      pd.bdate_range(
          df.index.min().date(),
          df.index.max().date(),
          tz=tz(),
      )
  )



Creating adjacent subplots https://matplotlib.org/stable/gallery/subplots_axes_and_figures/ganged_plots.html



In [ ]:
def plot_daily_profile(df:pd.DataFrame, resolution_usd:float=resolution_usd):
  dates = get_dates_tuple(df)

  fig, axs = plt.subplots(nrows=1, ncols=len(dates), figsize=(16,9), sharey=True)
  fig.subplots_adjust(wspace=0)

  bins_historical = get_bins(df["Close"], resolution_usd)

  ax_collection = []
  for date, ax in zip(dates, axs):
    ax_collection.append(
      plot_candlestick_vprofile(df_one_day(df, date), ax=ax, bins=bins_historical)
    )
  return ax_collection



In [ ]:
plot_daily_profile(spy_historical);

